In [35]:
import time
from datetime import datetime, timezone
from dateutil import parser

import requests

In [36]:
workspace_id = notebookutils.runtime.context["currentWorkspaceId"]
headers = {"Authorization": f"Bearer {notebookutils.credentials.getToken('pbi')}"}

## Get the folder id in which everything is deployed

In [38]:
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/folders"
response = requests.get(url, headers=headers)
response.raise_for_status()
folder_id = next(item["id"] for item in response.json()["value"] if item["displayName"] == "acr")
print(folder_id)

814285b5-7dd6-427e-b5ec-89c04137a2b9


## Force refresh of SQL analytics endpoint

In [39]:
# Get sql_endpoint_id
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/sqlEndpoints"
response = requests.get(url, headers=headers)
response.raise_for_status()
sql_endpoint_id = next(item["id"] for item in response.json()["value"] if "acr" in item["displayName"] and folder_id == item.get("folderId"))

# Force refresh sql endpoint
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/sqlEndpoints/{sql_endpoint_id}/refreshMetadata"
response = requests.post(url, headers=headers, json={})
response.raise_for_status()

if response.status_code == 200:
    print(response.json())

{'value': [{'tableName': 'acr', 'status': 'NotRun', 'startDateTime': '2025-09-12T15:25:16.9235788Z', 'endDateTime': '2025-09-12T15:25:16.9704539Z', 'lastSuccessfulSyncDateTime': '2025-09-12T15:16:35.2294715Z'}]}


## wait to ensure refresh is completed

In [40]:
if response.status_code == 202:
    end_datetime_str = response.json()['value'][0]['endDateTime']

    # Parse the endDateTime to a datetime object (UTC)
    end_datetime = parser.isoparse(end_datetime_str)

    # Get current UTC time
    now = datetime.now(timezone.utc)

    # Calculate how many seconds to wait
    wait_seconds = (end_datetime - now).total_seconds()

    # If wait_seconds is positive, sleep; otherwise, continue immediately
    if wait_seconds > 0:
        print(f"Waiting for {wait_seconds:.2f} seconds until {end_datetime} before next request...")
        time.sleep(wait_seconds)
    else:
        print("endDateTime is in the past.")
        
print("Let's wait an additional 30 seconds to be sure.")        
time.sleep(30)

Let's wait an additional 30 seconds to be sure.


## Refresh semantic model (which is based on SQL AE)

In [41]:
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/semanticModels"
response = requests.get(url, headers=headers)
response.raise_for_status()
semantic_model_id = [item["id"] for item in response.json()["value"] if item.get("folderId") == folder_id]

In [42]:
if semantic_model_id:
    headers = {"Authorization": f"Bearer {notebookutils.credentials.getToken('pbi')}"}
    response = requests.post(f"https://api.powerbi.com/v1.0/myorg/datasets/{semantic_model_id}/refreshes", headers=headers)
    response.raise_for_status()
    if response.status_code == 202:
        refresh_status_url = f"https://api.powerbi.com/v1.0/myorg/datasets/{semantic_model_id}/refreshes"
        status_response = requests.get(refresh_status_url, headers=headers)
        refreshes = status_response.json()

        # Look for your specific refresh using the RequestId
        for refresh in refreshes['value']:
            if refresh.get('requestId') == response.headers["RequestId"]:
                print(f"Status: {refresh['status']}")
                print(f"Start Time: {refresh.get('startTime', 'N/A')}")
                print(f"End Time: {refresh.get('endTime', 'N/A')}")
                break
    else:
        print(response.json())
else:
    print(f"⚠️ no semantic model found for the specified folder!")

⚠️ no semantic model found for the specified folder!
